In [1]:
import random
from keras.preprocessing import sequence
import tensorflow as tf
import numpy as np
import re

Using TensorFlow backend.


In [2]:

def only_lowercase_words(text): return re.findall(r'\w+', text.lower())

words = only_lowercase_words(open('big.txt').read())
MAX_LENGTH_WORD = 10
words = [w for w in words if len(w) <= MAX_LENGTH_WORD]
print(len(words))


words = words[:1000]
Xdata = []
Ydata = []



feature_dict= dict()
feature_list = list()

PADDING_CHARACTER = '~'
feature_dict[PADDING_CHARACTER]=0
feature_list.append(PADDING_CHARACTER)
max_features = 1

def get_vector_from_string(input_s):
    global max_features
    vector_x = []
    for i in input_s:
        if i not in feature_dict:
            feature_dict[i]=max_features
            feature_list.append(i)
            max_features += 1
        vector_x.append(feature_dict[i])
    return vector_x
def add_to_data(input_s,output_s):
    if len(input_s) < MAX_LENGTH_WORD and len(output_s) < MAX_LENGTH_WORD:
        vector_x = get_vector_from_string(input_s)
        vector_y = get_vector_from_string(output_s)
        Xdata.append(vector_x)
        Ydata.append(vector_y)

        
def print_vector(vector,end_token='\n'):
    print(''.join([feature_list[i] for i in vector]),end=end_token)

for line in words:
    in_s,out_s = line,line
    add_to_data(in_s,out_s)
for i in range(10):
    print_vector(Xdata[i],end_token='')
    print(' -> ', end='')
    print_vector(Ydata[i])
    

1088098
the -> the
project -> project
gutenberg -> gutenberg
ebook -> ebook
of -> of
the -> the
of -> of
sherlock -> sherlock
holmes -> holmes
by -> by


In [3]:

before_padding = Xdata[0]
Xdata = sequence.pad_sequences(Xdata, maxlen=MAX_LENGTH_WORD)
Ydata = sequence.pad_sequences(Ydata, maxlen=MAX_LENGTH_WORD)
after_padding  = Xdata[0]

print_vector(before_padding,end_token='')
print(" -> after padding: ", end='')
print_vector(after_padding)

class DataSplitter:
    def __init__(self,percentage):
        self.percentage = percentage
    def split_data(self,data):
        splitpoint = int(len(data)*self.percentage)
        return data[:splitpoint], data[splitpoint:]
splitter = DataSplitter(0.8)
Xdata,Xtest = splitter.split_data(Xdata)
Ydata,Ytest = splitter.split_data(Ydata)

def get_random_reversed_dataset(Xdata,Ydata,batch_size):
    newX = []
    newY = []
    for _ in range(batch_size):
        index_taken = random.randint(0,len(Xdata)-1)
        newX.append(Xdata[index_taken])
        newY.append(Ydata[index_taken][::-1])
    return newX,newY

the -> after padding: ~~~~~~~the


In [4]:


batch_size = 64

embedding_dim = 32
letters_in_alphabet = 26
memory_dim = 248

enc_input = [tf.placeholder(tf.int32, shape=(None,)) for i in range(MAX_LENGTH_WORD)]
one_hot_input = [tf.one_hot(a, letters_in_alphabet,axis=-1) for a in enc_input]
dec_output = [tf.placeholder(tf.int32, shape=(None,)) for t in range(MAX_LENGTH_WORD)]
one_hot_dec_output = [tf.one_hot(a, letters_in_alphabet,axis=-1) for a in dec_output]

decoder_input = ([tf.zeros_like(one_hot_dec_output[0], dtype=tf.float32)]+[one_hot_dec_output[t] for t in range(MAX_LENGTH_WORD-1)])


cell = tf.nn.rnn_cell.GRUCell(memory_dim)

initial_state = cell.zero_state(batch_size, tf.float32)
output, enc_state = tf.nn.rnn(cell=cell,inputs=one_hot_input,dtype=tf.float32,initial_state=initial_state)

softmax_w = tf.Variable(tf.truncated_normal([memory_dim,letters_in_alphabet],stddev=0.5))
softmax_b = tf.Variable(tf.truncated_normal([letters_in_alphabet],stddev=0.5))
decoder_cell = tf.nn.rnn_cell.GRUCell(memory_dim)


def loop(prev, _):
    prev = tf.matmul(prev, softmax_w) + softmax_b
    prev_symbol = tf.stop_gradient(tf.argmax(prev, 1))
    return tf.one_hot(prev_symbol,letters_in_alphabet)#tf.nn.embedding_lookup(embedding, prev_symbol)


strange_weights = tf.Variable(tf.truncated_normal([memory_dim,letters_in_alphabet],stddev=0.5))
def stupid_loop_function(prev,some_index):
    return prev

with tf.variable_scope("decoder1") as scope:
    outputs, decoderstate = tf.nn.seq2seq.rnn_decoder(decoder_input, enc_state, decoder_cell)
with tf.variable_scope("decoder1",reuse=True) as scope:
    testoutputs, teststate = tf.nn.seq2seq.rnn_decoder(decoder_input, enc_state, decoder_cell,loop_function=loop)

output_unstacked = tf.unpack(outputs)
output_multiplied = [tf.matmul(a,softmax_w) + softmax_b for a in output_unstacked]

weights = [tf.ones([batch_size, letters_in_alphabet], dtype=tf.float32) for labels_t in dec_output]

print(dec_output)
print(weights)
print(output_multiplied)
difference = [a-b for a in output_multiplied for b in one_hot_dec_output]

loss = sum([tf.nn.l2_loss(a) for a in difference])
#loss = tf.nn.seq2seq.sequence_loss(output_multiplied, dec_output, weights, letters_in_alphabet)

optimizer = tf.train.AdamOptimizer()
train_op = optimizer.minimize(loss)

# Init everything
sess = tf.InteractiveSession()
sess.run(tf.initialize_all_variables())



[<tf.Tensor 'Placeholder_10:0' shape=(?,) dtype=int32>, <tf.Tensor 'Placeholder_11:0' shape=(?,) dtype=int32>, <tf.Tensor 'Placeholder_12:0' shape=(?,) dtype=int32>, <tf.Tensor 'Placeholder_13:0' shape=(?,) dtype=int32>, <tf.Tensor 'Placeholder_14:0' shape=(?,) dtype=int32>, <tf.Tensor 'Placeholder_15:0' shape=(?,) dtype=int32>, <tf.Tensor 'Placeholder_16:0' shape=(?,) dtype=int32>, <tf.Tensor 'Placeholder_17:0' shape=(?,) dtype=int32>, <tf.Tensor 'Placeholder_18:0' shape=(?,) dtype=int32>, <tf.Tensor 'Placeholder_19:0' shape=(?,) dtype=int32>]
[<tf.Tensor 'ones:0' shape=(64, 26) dtype=float32>, <tf.Tensor 'ones_1:0' shape=(64, 26) dtype=float32>, <tf.Tensor 'ones_2:0' shape=(64, 26) dtype=float32>, <tf.Tensor 'ones_3:0' shape=(64, 26) dtype=float32>, <tf.Tensor 'ones_4:0' shape=(64, 26) dtype=float32>, <tf.Tensor 'ones_5:0' shape=(64, 26) dtype=float32>, <tf.Tensor 'ones_6:0' shape=(64, 26) dtype=float32>, <tf.Tensor 'ones_7:0' shape=(64, 26) dtype=float32>, <tf.Tensor 'ones_8:0' shap

In [5]:
for i in range(100000):
    Xin,Yin = get_random_reversed_dataset(Xdata,Ydata,batch_size)
    Xin = np.array(Xin).T
    Yin = np.array(Yin).T
    feed_dict = {enc_input[t]: Xin[t] for t in range(MAX_LENGTH_WORD)}
    feed_dict.update({dec_output[t]: Yin[t] for t in range(MAX_LENGTH_WORD)})
    _, my_enc_state, my_outputs, my_output_multiplied,my_loss = sess.run([train_op,enc_state,outputs,output_multiplied,loss],feed_dict)
    if i%250==1:
        print(my_loss)
# print(my_enc_state)
# print(len(my_enc_state))
# print(len(my_enc_state[0]))
# print(my_enc_state[0])
# print('expected above: 64 x 248')
print(len(my_outputs))
print(len(my_outputs[0]))
# print(len(my_outputs[0][0]))
# print(my_outputs[0][0])
print("multiplied")
print(len(my_output_multiplied))
print(len(my_output_multiplied[0]))
print(len(my_output_multiplied[0][0]))

# for index_now in range(10):

#     
#     _, l,the_state,ohin,sfsf,the_outputs,unstacked_sdfsf = sess.run([train_op,loss,state,one_hot_input,outputs,
#                                                      output_multiplied,output_unstacked], feed_dict)
#     if index_now%100==1:
#         print(l)
#         print(the_state)
#         print(len(the_outputs))
#         print(len(the_outputs[0]))
#         print(len(the_outputs[0][0]))
#         print(the_outputs[0][0])
#         print(unstacked_sdfsf)
#         print(the_state)
#         print(len(the_state))
#         print(len(the_state[0]))
#         for character in ohin:
#             print(character[0]) # get 0 from batch
        

29809.4
1922.89
1819.02
1748.89
1924.88
1809.92
1897.82
1903.2
1839.46
1898.41
1824.22
1806.95
1825.8
1903.08
1887.2
1801.6
1722.44
1886.61
1880.75
2055.13
1791.78
1794.29
1883.44
1848.32
1852.56
1775.53
1861.32
1953.35
1896.83
1788.54
1811.82
1853.85
1838.73
1873.38
1875.86
1872.34
1837.89
1796.14
1840.33
1921.85
1930.13
1804.81
2021.81
1857.37
1885.89
1763.71
1770.36
1930.4
1727.63
1915.64
1887.87
1861.14
1976.9
1835.2
1941.15
1900.59
1957.87
1755.52
2009.73
1962.76
1735.36
1928.34
1836.15
1884.51
1839.03
1876.73
1842.79
1896.62
1844.17
1792.17
1656.5
1876.83
1786.25
1867.97
1873.25
1818.84
1689.56
1794.36
1759.69
1707.3
1842.83
1872.08
1753.39
1835.2
1893.09
2022.3
1875.15
1822.36
1834.15
1849.19
1759.77
1834.55
1769.57
1851.57
1812.63
1733.55
1946.51
1923.09
1898.11
1743.65
1844.2
1707.59
1838.32
1909.56
1758.86
1847.05
1764.11
1767.62
1690.04
1884.3
1913.77
1826.12
1813.2
1938.2
1716.72
1836.34
1893.33
1821.84
1732.07
1787.05
1806.47
1739.59
1924.1
1851.12
1950.56
1793.03
1907.64


KeyboardInterrupt: 

In [8]:
tf.unpack?
